In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.set_option("display.max_rows",2000)
pd.set_option("display.max_column",2000)

df = pd.read_csv("train.csv")

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [42]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


Bir müşterinin birden fazla satırı (aldığı ürün) olabilir. (many to many) 

Her satır müşteri başına satılan ürünü temsil ediyor (multi index userid ve productid).

Volume de satır sayıları toplanarak bulundu, = satılan ürün sayısı

# Gender Column

Erkekler daha seyrek alışveriş yapmasına rağmen ortalama daha yüksek harcama yapıyor. 

(Volume: %300 daha fazla, Avg spending: %8 daha fazla)

In [43]:
sum_of_purchase = df.groupby(["Gender"]).sum()[["Purchase"]]
volume = df.groupby(["Gender"]).count()[["User_ID"]]
Grouped_by_Gender = pd.concat([sum_of_purchase, volume], axis=1)
Grouped_by_Gender.rename(columns={"User_ID": "Volume"}, inplace = True)

Grouped_by_Gender["Average_Spending_Per_Product"] = Grouped_by_Gender["Purchase"] / Grouped_by_Gender["Volume"]



In [44]:
Grouped_by_Gender

,Purchase,Volume,Average_Spending_Per_Product
Gender,,,
F,1186232642,135809,8734.565765
M,3909580100,414259,9437.526040


In [45]:
df_gender_category_purchase = df.groupby(["Gender", "Product_Category_1"]).sum()["Purchase"]

df_gender_category_volume =df.groupby(["Gender", "Product_Category_1"]).count()["User_ID"]

df_average_spending = df_gender_category_purchase / df_gender_category_volume
df_average_spending.sort_values(ascending=False)

Gender  Product_Category_1
F       10                    19692.076592
M       10                    19670.731264
F       7                     16394.853659
M       7                     16355.789777
        6                     15907.851009
F       9                     15724.314286
        6                     15596.428164
M       9                     15498.888235
        15                    14797.431350
        16                    14793.384056
F       15                    14695.326960
        16                    14681.491257
        14                    13747.362761
M       1                     13608.164721
F       1                     13597.162619
M       14                    12722.321111
F       2                     11407.496819
M       2                     11203.590520
F       3                     10262.656677
M       17                    10209.732558
        3                     10026.550081
F       17                     9846.403226
        8                  

# Gender Column

In [46]:
grouped_purchase = df.groupby(["Gender"]).sum()["Purchase"]
gender_grouped_volume = df.groupby(["Gender"]).count()["User_ID"]

gender_grouped = pd.concat([volume, pd.DataFrame(sum_of_purchase)], axis=1)
gender_grouped = gender_grouped.rename(columns={"User_ID": "Volume"})
gender_grouped["Average_Spending_Per_Product"] = gender_grouped["Purchase"] / gender_grouped["Volume"]
gender_grouped["Purchase_Ratio_by_Gender"] = 0
gender_grouped["Purchase_Ratio_by_Gender"].iloc[0] = gender_grouped["Purchase"].iloc[0] / (gender_grouped["Purchase"].iloc[0] + gender_grouped["Purchase"].iloc[1])
gender_grouped["Purchase_Ratio_by_Gender"].iloc[1] = gender_grouped["Purchase"].iloc[1] / (gender_grouped["Purchase"].iloc[0] + gender_grouped["Purchase"].iloc[1])
gender_grouped

c:\users\ugurz\miniconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Volume,Purchase,Average_Spending_Per_Product,Purchase_Ratio_by_Gender
Gender,,,,
F,135809,1186232642,8734.565765,0.232786
M,414259,3909580100,9437.526040,0.767214


# Product Category Column

In [1]:
#M'lerin yaptığı 9400'lük spending ortalama alışveriş miktarı değil, aldıkları ürünlerin ortalama fiyatı
#erkekler average spending olarak %8 daha fazla harcama yapıyorlar.
#purchase volumü olarak ise erkekler kadınlara oranla %330 oranında alışveriş yapıyorlar.  

In [2]:
df.head()

NameError: name 'df' is not defined

In [49]:
sold_product_category = df.groupby("Product_Category_1").count()["User_ID"]

sold_product_category.index

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
            20],
           dtype='int64', name='Product_Category_1')

In [50]:
sold_product_category = sold_product_category.sort_values(ascending=False)
x_axis = (sold_product_category.index.values).astype(str)

fig = px.bar(sold_product_category, x=x_axis, y=sold_product_category.values)
fig.show()

In [51]:
#en çok satılan kategorileri ifade eder. 5, 1 ve 8 kategorileri ilk üçte. 

In [52]:
data = [go.Bar(
   x = sold_product_category.index,
   y = sold_product_category.values,
     dx=1
)]

In [53]:
print(df.isnull().sum())

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            173638
Product_Category_3            383247
Purchase                           0
dtype: int64


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 12 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     550068 non-null  int64  
 1   Product_ID                  550068 non-null  object 
 2   Gender                      550068 non-null  object 
 3   Age                         550068 non-null  object 
 4   Occupation                  550068 non-null  int64  
 5   City_Category               550068 non-null  object 
 6   Stay_In_Current_City_Years  550068 non-null  object 
 7   Marital_Status              550068 non-null  int64  
 8   Product_Category_1          550068 non-null  int64  
 9   Product_Category_2          376430 non-null  float64
 10  Product_Category_3          166821 non-null  float64
 11  Purchase                    550068 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 50.4+ MB


In [55]:
#category1'de nan value yokken category2 ve category3'de daha fazla nan value var. 
#şöyle bir fikir var: acaba tek kategoriye sahip olan ürünler çok kategoriye sahip olan ürünlerden daha fazla mı satılıyor.
#bunu anlamak için nan valueları droplayıp bakalım. 

### Age groupby

In [56]:
sum_of_purchase = df.groupby(["Age"]).sum()[["Purchase"]]
volume = df.groupby(["Age"]).count()[["User_ID"]]
Grouped_by_Age = pd.concat([sum_of_purchase,volume], axis=1)
Grouped_by_Age.rename(columns={"User_ID": "Volume"}, inplace = True)

Grouped_by_Age["Average_Spending_Per_Product"] = Grouped_by_Age["Purchase"] / Grouped_by_Age["Volume"]

In [57]:
Grouped_by_Age

,Purchase,Volume,Average_Spending_Per_Product
Age,,,
0-17,134913183,15102,8933.464640
18-25,913848675,99660,9169.663606
26-35,2031770578,219587,9252.690633
36-45,1026569884,110013,9331.350695
46-50,420843403,45701,9208.625697
51-55,367099644,38501,9534.808031
55+,200767375,21504,9336.280459


In [58]:
#user'a yönelik en çok harcama yapanlar listesi oluştursak

# Occupation Column

In [59]:
#burada hangi mesleklerin ürün alma miktarlarının sıralaması

In [60]:
sum_of_purchase = df.groupby(["Occupation"]).sum()[["Purchase"]] 
volume = df.groupby(["Occupation"]).count()[["User_ID"]] 
Grouped_by_Occupation = pd.concat([sum_of_purchase,volume],axis=1)
Grouped_by_Occupation.rename(columns={"User_ID": "Volume"}, inplace = True)

In [61]:
Grouped_by_Occupation["Average_Spending_Per_Product"] = 0
Grouped_by_Occupation["Average_Spending_Per_Product"] = Grouped_by_Occupation["Purchase"] / Grouped_by_Occupation["Volume"]

In [62]:
Grouped_by_Occupation

,Purchase,Volume,Average_Spending_Per_Product
Occupation,,,
0,635406958,69638,9124.428588
1,424614144,47426,8953.193270
2,238028583,26588,8952.481683
3,162002168,17650,9178.593088
4,666244484,72308,9213.980251
5,113649759,12177,9333.149298
6,188416784,20355,9256.535691
7,557371587,59133,9425.728223
8,14737388,1546,9532.592497


# City_Category

In [63]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [64]:
sum_of_purchase = df.groupby(["City_Category"]).sum()[["Purchase"]] 
volume = df.groupby(["City_Category"]).count()[["User_ID"]] 
Grouped_by_City_Category = pd.concat([sum_of_purchase,volume],axis=1)
Grouped_by_City_Category.rename(columns={"User_ID": "Volume"}, inplace = True)

In [65]:
Grouped_by_City_Category["Average_Spending_Per_Product"] = 0
Grouped_by_City_Category["Average_Spending_Per_Product"] = Grouped_by_City_Category["Purchase"] / Grouped_by_City_Category["Volume"]

In [66]:
Grouped_by_City_Category

,Purchase,Volume,Average_Spending_Per_Product
City_Category,,,
A,1316471661,147720,8911.939216
B,2115533605,231173,9151.300563
C,1663807476,171175,9719.920993


In [67]:
#Grouped_by_City_Category değişkenine bakıldığında en çok ürün B şehrinde satılırken ürün başına en yüksek fiyat C şehrinde
#ödenmiş. 

# Stay_In_Current_City_Years

In [68]:
sum_of_purchase = df.groupby(["Stay_In_Current_City_Years"]).sum()[["Purchase"]] 
volume = df.groupby(["Stay_In_Current_City_Years"]).count()[["User_ID"]] 
Grouped_by_Stay_In_Current_City_Years = pd.concat([sum_of_purchase,volume],axis=1)
Grouped_by_Stay_In_Current_City_Years.rename(columns={"User_ID": "Volume"}, inplace = True)

In [69]:
Grouped_by_Stay_In_Current_City_Years["Average_Spending_Per_Product"] = 0
Grouped_by_Stay_In_Current_City_Years["Average_Spending_Per_Product"] = Grouped_by_Stay_In_Current_City_Years["Purchase"] / Grouped_by_Stay_In_Current_City_Years["Volume"]

In [70]:
Grouped_by_Stay_In_Current_City_Years

,Purchase,Volume,Average_Spending_Per_Product
Stay_In_Current_City_Years,,,
0,682979229,74398,9180.075123
1,1792872533,193821,9250.145923
2,949173931,101838,9320.429810
3,884902659,95285,9286.904119
4+,785884390,84726,9275.598872


In [71]:
#şehirde 1 yıl kalanlar en fazla ürün alımını gerçekleştirmiş. 1 yıl kalanlar diğer kategoriler ile karşılaştırıldığında
#yaklaşık olarak 2 kata sahipler. Öte yandan alınan ürünlere ödenen ortalama fiyat konusunda kategoriler arasında çok 
#bir fark yok

# Marital_Status

In [72]:
sum_of_purchase = df.groupby(["Marital_Status"]).sum()[["Purchase"]] 
volume = df.groupby(["Marital_Status"]).count()[["User_ID"]] 
Grouped_by_Marital_Status = pd.concat([sum_of_purchase,volume],axis=1)
Grouped_by_Marital_Status.rename(columns={"User_ID": "Volume"}, inplace = True)

In [73]:
Grouped_by_Marital_Status["Average_Spending_Per_Product"] = 0
Grouped_by_Marital_Status["Average_Spending_Per_Product"] = Grouped_by_Marital_Status["Purchase"] / Grouped_by_Marital_Status["Volume"]

In [74]:
Grouped_by_Marital_Status.index = Grouped_by_Marital_Status.index.astype('str')

In [75]:
Grouped_by_Marital_Status

,Purchase,Volume,Average_Spending_Per_Product
Marital_Status,,,
0,3008927447,324731,9265.907619
1,2086885295,225337,9261.174574


In [76]:
#ürünlere ödenen ortalama fiyat konusunda çok bir farklılık göze çarpmıyor.

# Visualization

In [77]:
gender_grouped

,Volume,Purchase,Average_Spending_Per_Product,Purchase_Ratio_by_Gender
Gender,,,,
F,135809,1186232642,8734.565765,0.232786
M,414259,3909580100,9437.526040,0.767214


In [78]:
go.Bar(name='Average_Spending_Per_Product', x=Grouped_by_Gender.index, y=Grouped_by_Gender["Average_Spending_Per_Product"], yaxis='y', offsetgroup=1),

go.Bar(name='Purchase', x=Grouped_by_Gender.index, y=Grouped_by_Gender["Purchase"], yaxis='y')



Bar({
    'name': 'Purchase',
    'x': array(['F', 'M'], dtype=object),
    'y': array([1186232642, 3909580100], dtype=int64),
    'yaxis': 'y'
})

In [79]:
fig_avg_spending_by_gender = go.Figure(
    data=[
        go.Bar(name='Average_Spending_Per_Product', x=Grouped_by_Gender.index, y=Grouped_by_Gender["Average_Spending_Per_Product"], yaxis='y', offsetgroup=1),
    ]
)

fig_volume_by_gender = go.Figure(
    data=[
        go.Bar(name='Volume', x=Grouped_by_Gender.index, y=Grouped_by_Gender.Volume, yaxis='y2', offsetgroup=2)
    ]
)

# Change the bar mode
# fig.update_layout(barmode='group', template="plotly_white")
# fig2.update_layout(barmode='group', template="plotly_white")

In [80]:


fig_gender = make_subplots(rows=1, cols=2,
                   subplot_titles=("Average Spending per Product", "Total Purchase Amount"))

fig_gender.add_trace(
    go.Bar(name='Average Spending per Product', x=gender_grouped.index, y=gender_grouped["Average_Spending_Per_Product"], yaxis='y'),
    row=1, col=1
)

fig_gender.add_trace(
   go.Bar(name='Total Purchase Amount', x=gender_grouped.index, y=gender_grouped["Purchase"], yaxis='y'),
    row=1, col=2
)

fig_gender.update_layout(height=500, width=1000, title_text="Gender Analysis", template="plotly_white")
fig_gender.update_layout(showlegend=False)
fig_gender.show()

Erkekler ürün başına daha fazla para ödüyorlar (9400 vs 8700) ve daha çok ürün satın almışlar. Ürün başına ödenen miktarların benzer olduğu düşünülürse purchase'ın 4 kat çıkması tutarlı.

In [81]:
Grouped_by_Age

,Purchase,Volume,Average_Spending_Per_Product
Age,,,
0-17,134913183,15102,8933.464640
18-25,913848675,99660,9169.663606
26-35,2031770578,219587,9252.690633
36-45,1026569884,110013,9331.350695
46-50,420843403,45701,9208.625697
51-55,367099644,38501,9534.808031
55+,200767375,21504,9336.280459


In [82]:
df_list = [Grouped_by_Gender, Grouped_by_Age, Grouped_by_Occupation, Grouped_by_City_Category,
          Grouped_by_Stay_In_Current_City_Years, Grouped_by_Marital_Status]
title_list = ["Gender", "Age", "Occupation", "City Category", "Stay in Current City Years", "Marital Status"]

figure_list = []

for i in range(len(df_list)):
    fig = make_subplots(rows=1, cols=2,
                       subplot_titles=("Average Spending per Product", "Total Purchase Amount"))

    fig.add_trace(
        go.Bar(name='Average Spending per Product', x=df_list[i].index, y=df_list[i]["Average_Spending_Per_Product"], yaxis='y'),
        row=1, col=1
    )

    fig.add_trace(
       go.Bar(name='Total Purchase Amount', x=df_list[i].index, y=df_list[i]["Purchase"], yaxis='y'),
        row=1, col=2
    )

    fig.update_layout(height=500, width=1000, title_text= title_list[i] + " Analysis", template="plotly_white")
    fig.update_layout(showlegend=False)
    figure_list.append(fig)

In [83]:
figure_list

for i in figure_list:
    i.show()

1. En çok para harcayan müşteri listesi çıkarılabilir (kampanyalar için).


2. Şimdiye kadarki tüm analizler ürün üzerindendi (avg spending per product). Şimdi unique müşteri sayısından yola çıkarak avg spending per person gibi kişi başına analizler yapabiliriz. Aslında Number of sold products ile "number of people purchasing" grafiklerini yer değiştireceğiz sadece.


3. Altın müşteriler gibi yeni bir kategori uydurabiliriz. En niş hedef kitle bulunabilir (C şehri, 26-35 yaş aralığı, erkek, bekar, meslek kodu 7) (reklam vermek gibi düşünebiliriz)



In [84]:
df

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969
...,...,...,...,...,...,...,...,...,...,...,...,...
550063,1006033,P00372445,M,51-55,13,B,1,1,20,NaN,NaN,368
550064,1006035,P00375436,F,26-35,1,C,3,0,20,NaN,NaN,371
550065,1006036,P00375436,F,26-35,15,B,4+,1,20,NaN,NaN,137
550066,1006038,P00375436,F,55+,1,C,2,0,20,NaN,NaN,365


In [85]:
# Buradan devam

gender_mask_list = df["Gender"].unique()

gender_mask_list[0]


'F'

In [86]:
dfg = df.groupby("User_ID")
